损失函数

In [ ]:
import numpy as np


def mean_squared_error(y,t):
    return 0.5 * np.sum((y-t)**2)

# delta为防止log(0)
# def cross_entropy_error(y,t):
#     delta = 1e-7
#     return -np.sum(t * np.log(y + delta))
# mini-batch版本
def cross_entropy_error(y,t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
        
    batch_size = y.shape[0]
    return -np.sum(t * np.log(y + 1e-7))/batch_size

def cross_entropy_error_not_one_hot(y,t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
        
    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size),t]+ 1e-7))/batch_size

minibatch学习：神经网络的学习也是从训练数据中选出一批数据（称为mini-batch,小批量），然后对每个mini-batch进行学习。比如，从60000个训练数据中随机选择100笔，再用这100笔数据进行学习。这种学习方式称为mini-batch学习

In [ ]:
import sys, os
sys.path.append(os.pardir)  # 为了导入父目录的文件而进行的设定
import numpy as np
from sample_code.dataset.mnist import load_mnist
from PIL import Image


def img_show(img):
    pil_img = Image.fromarray(np.uint8(img))
    pil_img.show()

(x_train, t_train), (x_test, t_test) = load_mnist(flatten=True, normalize=False)

train_size = x_train.shape[0]
batch_size = 10
batch_mask = np.random.choice(train_size,batch_size)
x_batch = x_train[batch_mask]
t_batch = t_train[batch_mask]
print(x_batch,t_batch)
print(batch_mask)

ndim输出的是维度，还是和形状shape不一样的

In [ ]:
t = np.array([[1,2],[4,5],[8,9]])
print(t.size)
print(t.ndim)
print(t.shape)

数值微分

In [ ]:
# 导数
import numpy as np
import matplotlib.pylab as plt
def numerical_diff(f,x):
    h = 1e-5
    return (f(x+h)-f(x-h)) / (2*h)

def function_1(x):
    return 0.01*x**2 + 0.1*x
def function_2(x):
    return x[0] ** 2 + x[1] ** 2 + x[2] ** 3
x = np.arange(0.0,20.0,0.1)
y = function_1(x)
plt.xlabel("x")
plt.ylabel("f(x)")
plt.plot(x,y)
plt.show()

print(numerical_diff(function_1,5))
print(numerical_diff(function_1,10))

梯度 gradient: 梯度表示的是各点处的函数值减小最多的方向
鞍点是从某个方向上看是极大值，从另一个方向上看则是极小值的点。

In [ ]:
def numerical_gradient(f,x):
    h = 1e-5
    grad = np.zeros_like(x)
    print(x,x.size)
    for idx in range(x.size):
        tmp_val = x[idx]
        
        x[idx] = tmp_val + h
        fxh1 = f(x)
        
        x[idx] = tmp_val - h
        fxh2 = f(x)
        
        grad[idx] = (fxh1 - fxh2) / (2*h)
        x[idx] = tmp_val # 还原值
    return grad

res1 = numerical_gradient(function_2, np.array([3.0,4.0,5.0]))
print(res1)
res1 = numerical_gradient(function_2, np.array([0.0,2.0,5.0]))
print(res1)
res1 = numerical_gradient(function_2, np.array([3.0,0.0,5.0]))
print(res1)

梯度下降法

In [ ]:
def gradient_descent(f, init_x, lr = 0.01, step_num = 100):
    x = init_x
    
    for i in range(step_num):
        grad = numerical_gradient(f,x)
        x -= lr* grad
    return x
init_x = np.array([-3.0,4.0,6.0])
res = gradient_descent(function_2, init_x=init_x, lr = 0.001, step_num=100)
print(res)

神经网络的梯度

In [ ]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from sample_code.common.functions import softmax, cross_entropy_error
from sample_code.common.gradient import numerical_gradient
class simpleNet:
    def __init__(self):
        self.W = np.random.randn(2,3) # 用高斯分布进行初始化
    def predict(self, x):
        return np.dot(x, self.W)
    def loss(self, x, t):
        z = self.predict(x)
        y = softmax(z)
        loss = cross_entropy_error(y, t)
        return loss

In [ ]:
net = simpleNet()
print(net.W)
x = np.array([0.6,0.9])
p = net.predict(x)
print(p)
print(np.argmax(p))
t = np.array([0,0,1])
net.loss(x,t)

In [ ]:
def f(W):
    return net.loss(x,t)
dW = numerical_gradient(f,net.W) #这里使用nditer，改变了net中的W本身，因此loss也随之变化
print(dW)

#使用lambda的版本：
f = lambda w: net.loss(x, t)
dW = numerical_gradient(f, net.W)

SGD：对随机选择的数据进行的梯度下降法

In [ ]:
import sys, os
sys.path.append(os.pardir)
from sample_code.common.functions import *
from sample_code.common.gradient import numerical_gradient

class TwoLayerNet:
    
    def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01):
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size,hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size,output_size)
        self.params['b2'] = np.zeros(output_size)
        
    def predict(self,x):
        W1,W2 = self.params['W1'], self.params['W2']
        b1,b2 = self.params['b1'], self.params['b2']
        
        a1 = np.dot(x,W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1,W2) + b2
        z2 = sigmoid(a2)
        y = softmax(z2)
        
        return y
    def loss(self,x,t):
        y = self.predict(x)
        return cross_entropy_error(y,t)
    
    def accuracy(self, x ,t):
        y = self.predict(x)
        y = np.argmax(y, axis = 1)
        t = np.argmax(t, axis = 1)
        
        accuracy = np.sum(y==t) / float(x.shape[0])
        return accuracy
    
    def numerical_gradient(self,x,t):
        loss_W = lambda W: self.loss(x,t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        return grads
    def gradient(self, x, t):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
        grads = {}
        
        batch_num = x.shape[0]
        
        # forward
        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)
        
        # backward
        dy = (y - t) / batch_num
        grads['W2'] = np.dot(z1.T, dy)
        grads['b2'] = np.sum(dy, axis=0)
        
        da1 = np.dot(dy, W2.T)
        dz1 = sigmoid_grad(a1) * da1
        grads['W1'] = np.dot(x.T, dz1)
        grads['b1'] = np.sum(dz1, axis=0)

        return grads
        

使用两层网络

In [49]:
import numpy as np
from sample_code.dataset.mnist import load_mnist
# from sample_code.ch04.two_layer_net import TwoLayerNet
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label = True)
train_loss_list = []
train_acc_list = []
test_acc_list = []
# 平均每个epoch的重复次数
iter_per_epoch = max(train_size / batch_size, 1)
# 超参数
iters_num = 100
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)
for i in range(iters_num):
    print(i)
    # 获取mini-batch
    batch_mask = np.random.choice(train_size, batch_size)
    print(batch_mask.shape)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    # 计算梯度
    grad = network.numerical_gradient(x_batch, t_batch)
    # grad = network.gradient(x_batch, t_batch) # 高速版!
    # 更新参数
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
    # 记录学习过程
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    # 计算每个epoch的识别精度
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print("train acc, test acc | " + str(train_acc) + ", " + str(test_acc))

0
(100,)
train acc, test acc | 0.0993, 0.1032
1
(100,)
2
(100,)
3
(100,)
4
(100,)
5
(100,)
6
(100,)
7
(100,)
8
(100,)
9
(100,)
10
(100,)
11
(100,)
12
(100,)
13
(100,)
14
(100,)
15
(100,)


KeyboardInterrupt: 